# Nuclear segmentation with emerin ring and spot detection

Scripts to detect nuclei from 4d images (z,x,y,c) where channel 0 is green spots and channel 1 is red emerin rings.
Mostly based on scripts from Lucien Hinderling, with some modifications and cleanup by Jennifer Semple.

Nuclear segmentation carried out with Cellpose.
Spot detection carried out with pytrack.

**Inputs**:

raw_input_path and denoised_input_path for directories containing raw and denoised images and output_path where results will be put. raw_input_path is used to create a dataframe with paths to images and the following 
columns:

*filename	date	experiment	strain	protein	id  raw_filepath    denoised_filepath*

example line:

*20240915_1268_E_bean_15um	20240915	3d	1268	DPY27	DPY27_3d_20240915_1268_E_bean_15um	/mnt/external.data/MeisterLab/Dario/Imaging/DP...	/mnt/external.data/MeisterLab/Dario/Imaging/DP...*

the id column is used to name images in the output_path directories

**Outputs**:

segmentation masks (.tif files) in output_path/segementation/

distance masks (.tif files) in output_path/edt/

nuclear measurements from middle slice of each nucleus (.csv files) in output_path/nuclei/

intensity measurements for nuclei with arrays of intensity/distance from middle slice of each nuclear mask (.pkl files) in output_path/dist/ 

qc plots of segmentation on original image (segmentation_XXX.pdf), individual masked nuclei (cropped_nuclei_XXX.pdf) in output_path/qc/

spot detection (.csv files, doesn't work very well) in output_path/spots/ with some qc in output_path/qc/spots*.pdf and spotGMM*.pdf




In [1]:
import napari
import torch
from skimage.measure import regionprops_table, regionprops
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import trackpy as tp
import cellpose
from cellpose import models
import edt
import glob
import os
import tqdm
from matplotlib_scalebar.scalebar import ScaleBar
import gc
import seaborn as sns
from sklearn.mixture import GaussianMixture
from bioio import BioImage
import bioio_nd2
import bioio_tifffile
from bioio.writers import OmeTiffWriter


anisotropy = (3,1,1) # Relative scale of (Z,X,Y) axes

nucChannel = 0 # red emerin rings
spotChannel = 1 # green spots

# in lucien's original scripts:
# channel 0 is green spots
# channel 1 is red emerin

In [2]:
# on server
raw_input_path = '/mnt/external.data/MeisterLab/Dario/Imaging/DPY27/1268/20240915_3d/'
denoised_input_path = raw_input_path+'N2V_1268RG_1273RG/denoised/'
output_path = '/mnt/external.data/MeisterLab/jsemple/lhinder/segmentation_Dario/DPY27/'

# on local machine (mac) with izbkingston mounted
#raw_input_path = '/Volumes/external.data/MeisterLab/Dario/Imaging/DPY27/1268/20240915_3d/'
#denoised_input_path = raw_input_path+'N2V_1268RG_1273RG/denoised/'
#output_path = '/Volumes/external.data/MeisterLab/jsemple/lhinder/segmentation_Dario/DPY27/'

if not os.path.exists(output_path+"/qc"):
    os.makedirs(output_path+"/qc")

if not os.path.exists(output_path+"/segmentation"):
    os.makedirs(output_path+"/segmentation")

if not os.path.exists(output_path+"/edt"):
    os.makedirs(output_path+"/edt")

if not os.path.exists(output_path+"/spots"):
    os.makedirs(output_path+"/spots")

if not os.path.exists(output_path+"/nuclei"):
    os.makedirs(output_path+"/nuclei")

if not os.path.exists(output_path+"/dist"):
    os.makedirs(output_path+"/dist")


raw_file_name_pattern = "/*.nd2"
denoised_file_name_pattern = "/*_n2v.tif"
raw_filepaths = sorted(glob.glob(raw_input_path + raw_file_name_pattern,recursive=True))
print(f"Found {len(raw_filepaths)} *.nd2 files.")

Found 24 *.nd2 files.


Generate data frame of file paths with metadata

In [3]:
df = pd.DataFrame()
df['filename'] = [os.path.basename(filepath)[:-4] for filepath in raw_filepaths]
tmpdate = [os.path.normpath(filepath).split(os.sep)[-2] for filepath in raw_filepaths]
df['date'] = pd.Series([exp.split('_')[0] for exp in tmpdate])
df['experiment'] = pd.Series([exp.split('_')[1] for exp in tmpdate])
df['strain'] = [os.path.normpath(filepath).split(os.sep)[-3] for filepath in raw_filepaths]
df['protein'] = [os.path.normpath(filepath).split(os.sep)[-4] for filepath in raw_filepaths]
df['id'] = df['protein'] + '_' + df['experiment'] + '_' + df['filename'] 
df['raw_filepath'] = raw_filepaths
df['denoised_filepath'] = [denoised_input_path+filename+'_n2v.tif' for filename in df['filename']]

df.to_csv(output_path+'fileList.csv',index=False)
df

,filename,date,experiment,strain,protein,id,raw_filepath,denoised_filepath
0,20240915_1268_E_bean_15um,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_bean_15um,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
1,20240915_1268_E_bean_15um_02,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_bean_15um_02,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
2,20240915_1268_E_early_15um,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
3,20240915_1268_E_early_15um_02,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_02,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
4,20240915_1268_E_early_15um_03,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_03,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
5,20240915_1268_E_early_15um_04,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_04,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
6,20240915_1268_E_early_15um_05,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_05,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
7,20240915_1268_E_early_15um_06,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_06,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
8,20240915_1268_E_early_15um_07,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_early_15um_07,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...
9,20240915_1268_E_late_15um,20240915,3d,1268,DPY27,DPY27_3d_20240915_1268_E_late_15um,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...


Load model

In [92]:
torch.cuda.device(0)

# using gpu, on server
#model = models.CellposeModel(pretrained_model='/mnt/external.data/MeisterLab/lhinder/segmentation_3d_anja/code/worms_1000epochs_v0', gpu=False, device =torch.device('cuda:0'))

# no gpu, on server
model = models.CellposeModel(pretrained_model='/mnt/external.data/MeisterLab/lhinder/segmentation_3d_anja/code/worms_1000epochs_v0')

# no gpu, from local machine with izbkingston mounted 
#model = models.CellposeModel(pretrained_model='/Volumes/external.data/MeisterLab/lhinder/segmentation_3d_anja/code/worms_1000epochs_v0')

/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/cellpose/resnet_torch.py:280: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(fi

## Functions for nuclear segmentation and qc

In [93]:
# Disable do_3D, there is a bug. 2D and stitching with overlap works much better.
# Takes around 7min for the whole image on the macbook
def segment_nuclei(img, model):
    ''' use pytorch cellpose model to segment nuclei'''
    masks,flows,styles = model.eval(img,do_3D=False,stitch_threshold=0.3,cellprob_threshold =0,diameter =36)
    return masks,flows,styles


def calc_distance_mask(masks,anisotropy):
    '''Calculate the distance map from the nuclei-edge towards the center of nucleus'''
    masks_edt = edt.edt(masks,anisotropy = anisotropy)
    return masks_edt


# 

def plot_qc_nuclei_crop(df, index, df_region_props, img, display = False, seed=1):
    '''Plot a cropped region of a random sample of 10 nuclei from each image'''
    nb_nuc = 10
    np.random.seed(seed)
    indices_to_sample = np.random.choice(range(len(df_region_props)),size = nb_nuc,replace = False)
    # sort indeces in descending order of area

    widths=[df_region_props['image'][i].shape[1] for i in indices_to_sample]

    fig, axs = plt.subplots(nrows = 2, ncols = nb_nuc, figsize = (15,5),dpi = 250, 
                            sharex=False, sharey=False, width_ratios=widths)
    fig.suptitle(f'Cropped nuclei {df.id.iloc[index]}', fontsize=16)

    for i,sample in enumerate(indices_to_sample):
        intensity_image = df_region_props['intensity_image'][sample][:,:,:,spotChannel] #show first spot channel
        image = df_region_props['image'][sample]
        mx = np.ma.masked_array(intensity_image,mask = ~image)
        z_height = image.shape[0] 
        axs[0,i].imshow(mx[int(z_height/2)])
        axs[0,i].spines['top'].set_visible(False)
        axs[0,i].spines['right'].set_visible(False)
        axs[0,i].spines['bottom'].set_visible(False)
        axs[0,i].spines['left'].set_visible(False)
        axs[0,i].get_xaxis().set_ticks([])
        axs[0,i].get_yaxis().set_ticks([])

    for i,sample in enumerate(indices_to_sample):
        intensity_image = df_region_props['intensity_image'][sample][:,:,:,nucChannel] #show second nuclear channel
        image = df_region_props['image'][sample]
        mx = np.ma.masked_array(intensity_image,mask = ~image)
        z_height = image.shape[0]
        axs[1,i].imshow(mx[int(z_height/2)])
        axs[1,i].spines['top'].set_visible(False)
        axs[1,i].spines['right'].set_visible(False)
        axs[1,i].spines['bottom'].set_visible(False)
        axs[1,i].spines['left'].set_visible(False)
        axs[1,i].get_xaxis().set_ticks([])
        axs[1,i].get_yaxis().set_ticks([])

        if i == nb_nuc-1:
            scalebar = ScaleBar(0.065, "um", length_fraction=1, box_alpha=0.7,color='black',location='lower right',height_fraction = 0.05,border_pad =-1)
            axs[1,i].add_artist(scalebar)

    #plt.tight_layout()
    fig.savefig(output_path + 'qc/cropped_nuclei_'+df.id.iloc[index]+'.pdf')
    if display == False:
        plt.close()
    else:
        plt.show()


def plot_single_nucleus_crop(df, index, df_region_props, nuc_index, img):
    '''Plot a cropped region of a particular nucleus'''

    fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (3,1.5),dpi = 250, sharey=True)
    fig.suptitle(f'{df.id.iloc[index]}', fontsize=6)

    intensity_image = df_region_props['intensity_image'][nuc_index][:,:,:,spotChannel] #show first spot channel
    image = df_region_props['image'][nuc_index]
    mx = np.ma.masked_array(intensity_image, mask = ~image)
    z_height = image.shape[0] 
    axs[0].imshow(mx[int(z_height/2)])
    axs[0].spines['top'].set_visible(False)
    axs[0].spines['right'].set_visible(False)
    axs[0].spines['bottom'].set_visible(False)
    axs[0].spines['left'].set_visible(False)
    axs[0].get_xaxis().set_ticks([])
    axs[0].get_yaxis().set_ticks([])


    intensity_image = df_region_props['intensity_image'][nuc_index][:,:,:,nucChannel] #show second nuclear channel
    image = df_region_props['image'][nuc_index]
    mx = np.ma.masked_array(intensity_image, mask = ~image)
    z_height = image.shape[0]
    axs[1].imshow(mx[int(z_height/2)])
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['right'].set_visible(False)
    axs[1].spines['bottom'].set_visible(False)
    axs[1].spines['left'].set_visible(False)
    axs[1].get_xaxis().set_ticks([])
    axs[1].get_yaxis().set_ticks([])


    scalebar = ScaleBar(0.065, "um", length_fraction=1, box_alpha=0.7,color='black',location='lower right',height_fraction = 0.05,border_pad =-1)
    axs[1].add_artist(scalebar)

    plt.show()


def plot_qc_segmentation_xyz(img, masks, index, df, display_plot=False, plotContours=False):
    '''Plot a 2x3 grid of xy, xz, yz slices of the image and the corresponding segmentation'''
    nucChannel = 0
    num_z=img.shape[1]
    num_y=img.shape[2]
    num_x=img.shape[3]
    nlabel=100

    fig = plt.figure(layout='constrained',dpi=450,figsize = (10,10))
    fig.suptitle(f'Segmentation for {df.id.iloc[index]}', fontsize=10)
    subfigs = fig.subfigures(2, 1, wspace=0.1)

    axsTop = subfigs[0].subplots(2, 3,sharex=True, sharey=True)
    #xy
    axsTop[0,0].imshow(label2rgb(masks[int(num_z*0.3),:,:],bg_label=0,bg_color=(255, 255, 255),colors=np.random.random((nlabel, 3))))
    axsTop[1,0].set_title('z='+str(int(num_z*0.3)), fontsize=8)
    axsTop[0,1].imshow(label2rgb(masks[int(num_z*0.5),:,:],bg_label=0,bg_color=(255, 255, 255),colors=np.random.random((nlabel, 3))))
    axsTop[1,1].set_title('z='+str(int(num_z*0.5)), fontsize=8)
    axsTop[0,2].imshow(label2rgb(masks[int(num_z*0.7),:,:],bg_label=0,bg_color=(255, 255, 255),colors=np.random.random((nlabel, 3))))
    axsTop[1,2].set_title('z='+str(int(num_z*0.7)), fontsize=8)

    axsTop[1,0].imshow(img[nucChannel,int(num_z*0.3),:,:],cmap = 'gray_r')
    axsTop[1,1].imshow(img[nucChannel,int(num_z*0.5),:,:],cmap = 'gray_r')
    axsTop[1,2].imshow(img[nucChannel,int(num_z*0.7),:,:],cmap = 'gray_r')

    if plotContours:
        axsTop[1,0].contour(masks[int(num_z*0.3),:,:], [0.5], linewidths=0.5, colors='r')
        axsTop[1,1].contour(masks[int(num_z*0.5),:,:], [0.5], linewidths=0.5, colors='r')
        axsTop[1,2].contour(masks[int(num_z*0.7),:,:], [0.5], linewidths=0.5, colors='r')



    for axss in axsTop:
        for ax in axss:
            #ax.set_xlim(0,num_x)
            #ax.set_ylim(0,num_y)
            ax.set_xticks([])
            ax.set_yticks([])

    axsBottom = subfigs[1].subplots(4, 3,sharex=True,sharey=True)
    #xz
    axsBottom[0,0].imshow(label2rgb(masks[:,int(num_y*0.3),:],bg_label=0,bg_color=(255, 255, 255),colors=np.random.random((nlabel, 3))))
    axsBottom[1,0].set_title('y='+str(int(num_y*0.3)), fontsize=8)
    axsBottom[0,1].imshow(label2rgb(masks[:,int(num_y*0.5),:],bg_label=0,bg_color=(255, 255, 255),colors=np.random.random((nlabel, 3))))
    axsBottom[1,1].set_title('y='+str(int(num_y*0.5)), fontsize=8)
    axsBottom[0,2].imshow(label2rgb(masks[:,int(num_y*0.7),:],bg_label=0,bg_color=(255, 255, 255),colors=np.random.random((nlabel, 3))))
    axsBottom[1,2].set_title('y='+str(int(num_y*0.7)), fontsize=8)

    axsBottom[1,0].imshow(img[nucChannel,:,int(num_y*0.3),:],cmap = 'gray_r')
    axsBottom[1,1].imshow(img[nucChannel,:,int(num_y*0.5),:],cmap = 'gray_r')
    axsBottom[1,2].imshow(img[nucChannel,:,int(num_y*0.7),:],cmap = 'gray_r')

    if plotContours:
        axsBottom[1,0].contour(masks[:,int(num_y*0.3),:], [0.5], linewidths=0.5, colors='r')
        axsBottom[1,1].contour(masks[:,int(num_y*0.5),:], [0.5], linewidths=0.5, colors='r')
        axsBottom[1,2].contour(masks[:,int(num_y*0.7),:], [0.5], linewidths=0.5, colors='r')


    #yz
    axsBottom[2,0].imshow(label2rgb(masks[:,:,int(num_x*0.3)],bg_label=0,bg_color=(255, 255, 255),colors=np.random.random((nlabel, 3))))
    axsBottom[3,0].set_title('x='+str(int(num_x*0.3)), fontsize=8)
    axsBottom[2,1].imshow(label2rgb(masks[:,:,int(num_x*0.5)],bg_label=0,bg_color=(255, 255, 255),colors=np.random.random((nlabel, 3))))
    axsBottom[3,1].set_title('x='+str(int(num_x*0.5)), fontsize=8)
    axsBottom[2,2].imshow(label2rgb(masks[:,:,int(num_x*0.7)],bg_label=0,bg_color=(255, 255, 255),colors=np.random.random((nlabel, 3))))
    axsBottom[3,2].set_title('x='+str(int(num_x*0.7)), fontsize=8)

    axsBottom[3,0].imshow(img[nucChannel,:,:,int(num_x*0.3)],cmap = 'gray_r')
    axsBottom[3,1].imshow(img[nucChannel,:,:,int(num_x*0.5)],cmap = 'gray_r')
    axsBottom[3,2].imshow(img[nucChannel,:,:,int(num_x*0.7)],cmap = 'gray_r')

    if plotContours:
        axsBottom[3,0].contour(masks[:,:,int(num_x*0.3)], [0.5], linewidths=0.5, colors='r')
        axsBottom[3,1].contour(masks[:,:,int(num_x*0.5)], [0.5], linewidths=0.5, colors='r')
        axsBottom[3,2].contour(masks[:,:,int(num_x*0.7)], [0.5], linewidths=0.5, colors='r')

    for axss in axsBottom:
        for ax in axss:
            #ax.set_ylim(0,num_z)
            ax.set_xticks([])
            ax.set_yticks([])

    plt.tight_layout()
    if display_plot:
        plt.show()
    else:
        fig.savefig(output_path + 'qc/segmentation_'+df.id.iloc[index]+'.pdf')
        plt.close()

In [94]:
## Run the segmentation script on all images (reserve more than 24GB!)
# this produces segmentation, segmentation_qc and edt files
def run_nuclear_segmentation(indices, df):
    '''Run the segmentation on all images in the dataframe'''
    for index in tqdm.tqdm(indices):
        # Load the data
        img = BioImage(df.denoised_filepath.iloc[index], reader=bioio_tifffile.Reader)
        img = img.get_image_data("CZYX", T=0)


        # Segment nuclei 
        masks,flows,styles = segment_nuclei(img[nucChannel,:,:,:],model) # Run the segmentation
        plot_qc_segmentation_xyz(img,masks,index, df, display_plot = False)                         # Create qc plot
        OmeTiffWriter.save(masks, output_path+'segmentation/'+df.id.iloc[index]+'.tif')

        del flows
        del styles
        gc.collect()
        
        # Calculate edt 
        masks_edt = calc_distance_mask(masks,anisotropy)
        OmeTiffWriter.save(masks_edt, output_path+'edt/'+df.id.iloc[index]+'.tif')

        del masks
        del masks_edt
        gc.collect()
        continue


In [95]:
## read images
## crop the nuclei slices
## calculate EDT transform
## for each nuclei loop over all distances (1:40) and take mean
## array of distance/intensity measurements are taken only for middle slice of mask (?)

## nucleus_id | nucleus volume | [1:20] mean intensities | group | ...


def run_dist_analysis(indices,df):
    '''Run the distance analysis on all images in the dataframe'''
    for index in tqdm.tqdm(indices):
        
        df_nuclei = pd.DataFrame()
        print(df.iloc[index].raw_filepath)
        
        masks = BioImage(output_path+'segmentation/'+df.id.iloc[index]+'.tif', reader=bioio_tifffile.Reader)
        masks = masks.get_image_data("ZYX", T=0, C=0)

        img = BioImage(df.raw_filepath.iloc[index], reader=bioio_nd2.Reader)
        img = img.get_image_data("ZYXC", T=0)
            
          
        df_region_props = regionprops_table(masks,img, properties = ['label', 'area','centroid','MajorAxisLength','solidity','image','intensity_image'])
        df_region_props = pd.DataFrame(df_region_props)

        if len(df_region_props)>=10:
            plot_qc_nuclei_crop(df, index, df_region_props, img, display = False) 

        for i in range(len(df_region_props)):
            df_nuclei_temp = pd.DataFrame()

            intensity_image_spots = df_region_props['intensity_image'][i][:,:,:,spotChannel] #show spot channel
            intensity_image_nuclei = df_region_props['intensity_image'][i][:,:,:,nucChannel] #show nuclear ring channel

            image = df_region_props['image'][i]  # binary 3d mask

            # Extract the intensity per distance
            mx_spots = np.ma.masked_array(intensity_image_spots, mask = ~image) # 3d masked spot channel
            mx_nuclei = np.ma.masked_array(intensity_image_nuclei,mask = ~image) # 3d masked nuclear ring channel
            mx_mask = np.ma.masked_array(image,mask = ~image)  # 3d masked binary mask

            z_height = image.shape[0]

            slice_spots = mx_spots[int(z_height/2)]
            slice_nuclei = mx_nuclei[int(z_height/2)]
            slice_mask = mx_mask[int(z_height/2)]

            slice_mask_edt = edt.edt(slice_mask)
            slice_mask_edt = np.ma.masked_array(slice_mask_edt, mask = ~(slice_mask_edt>0)) 

            results = regionprops_table(slice_mask_edt.astype('int'),slice_nuclei,properties=['label','intensity_mean'])
            intensity_dist_nuclei = results['intensity_mean']

            results = regionprops_table(slice_mask_edt.astype('int'),slice_spots,properties=['label','intensity_mean'])
            intensity_dist_spots = results['intensity_mean']

            dist = results['label']

            df_nuclei_temp['label']  = [df_region_props.label.iloc[i]]
            df_nuclei_temp['bb_dimZ']  = [mx_spots.shape[0]]
            df_nuclei_temp['bb_dimY']  = [mx_spots.shape[1]]
            df_nuclei_temp['bb_dimX']  = [mx_spots.shape[2]]
            df_nuclei_temp['centroid_z'] = df_region_props['centroid-0'][i]
            df_nuclei_temp['centroid_y'] = df_region_props['centroid-1'][i]
            df_nuclei_temp['centroid_x'] = df_region_props['centroid-2'][i]
            df_nuclei_temp['major_axis_length'] = df_region_props['MajorAxisLength'][i]
            df_nuclei_temp['solidity'] = df_region_props['solidity'][i]
            df_nuclei_temp['mean'] = [np.ma.mean(mx_spots)]
            df_nuclei_temp['median'] = [np.ma.median(mx_spots)]
            df_nuclei_temp['std']=  [np.ma.std(mx_spots)]
            df_nuclei_temp['sum']= [np.ma.sum(mx_spots)]
            df_nuclei_temp['variance']= [np.ma.var(mx_spots)]
            df_nuclei_temp['max'] = [np.ma.max(mx_spots)]
            df_nuclei_temp['min'] = [np.ma.min(mx_spots)]
            df_nuclei_temp['volume'] = [np.sum(np.invert(mx_spots.mask))]
            df_nuclei_temp['id'] = [df.id.iloc[index]]
            df_nuclei_temp['intensity_dist_nuclei'] = [intensity_dist_nuclei]  # this is the emerin ring channel intensity on central slice
            df_nuclei_temp['intensity_dist_spots'] = [intensity_dist_spots] # this is the spot channel but not actual detected spots
            df_nuclei_temp['intensity_dist'] = [dist]  # this is the distance from the edge of the nucleus
            df_nuclei_temp['zproj_spots'] = [np.max(intensity_image_spots[:,:,:], axis = 0)]
            df_nuclei_temp['zproj_nuclei'] = [np.max(intensity_image_nuclei[:,:,:], axis = 0)]

            df_nuclei = pd.concat([df_nuclei,df_nuclei_temp])

        # save as pickle because has array stored in Dataframe
        df_nuclei.to_pickle(output_path+'dist/'+df.id.iloc[index]+'.pkl') # Back up the DF for this FOV

        # save with metadata as csv for simple viewing 
        df_nuclei_for_csv = pd.merge(df_nuclei,df,on='id',how='left')
        df_nuclei_for_csv.drop( columns = [ 'intensity_dist_nuclei','intensity_dist_spots','intensity_dist' ], axis=1, inplace=True)
        df_nuclei_for_csv.to_csv(output_path+'nuclei/'+df.id.iloc[index]+'.csv', index=False)



## Functions for spot detection and qc

In [96]:
def find_spots(img, diameter=(9,5,5), separation=(3,2,2)):
    '''Find protein clusters of a certain size. Method is very sensitive, be sure to filter the spots afterwards using the signal value.
    Input: img -> 3D array from C0 containing
    Output: pandas.df containing data with all the detected spots.
            x,y,z: centroid of spot in image coordinates (px space)
            mass: total integrated brightness of the blob
            size: radius of gyration of its Gaussian-like profile
            ecc: eccentricity
            raw_mass: integrated brightness in raw_image '''

    #features = tp.locate(img, diameter=diameter, engine='numba',minmass = 10,percentile = 0.95,max_iterations=3,preprocess = True) #check 
    features = tp.locate(img[:,:,:], diameter, minmass=None, maxsize=None, separation=separation, noise_size=1, smoothing_size=None, threshold=None, invert=False, percentile=95, topn=None, preprocess=True, max_iterations=3, filter_before=None, filter_after=None, characterize=True, engine='numba')
    return features


def extract_spot_features(features, masks, masks_edt):
    '''For each spot get the label of the corresponding nucleus and distance to its envelope.'''

    for feature in features.iterrows():
        index = feature[0]
        x = round(feature[1].x)
        y = round(feature[1].y)
        z = round(feature[1].z)
        dist = masks_edt[z,y,x]
        label = masks[z,y,x]
        features.loc[index,'dist'] = dist
        features.loc[index,'label'] = int(label)
        
    return features
#features = extract_spot_features(features,masks,masks_edt)

def filter_spots(features, measure = 'signal', signal_strength = 0.1):
    '''Remove all spots that lie outside of a nucleus (or where nucleus is not detected.
       Remove all spots with signal<signal_strength'''
    features_filt = features[(features['dist']>0)&(features[measure]>signal_strength)]
    return features_filt

In [97]:
def plot_qc_spot_detection(index, df, df_features, img, display_plot = False,
                            measure = 'signal', thresholds = [1 ,2.5, 5, 10]):
    '''Plot the spot detection for a given image using several thresholds for the signal strength (shown on Z projections)'''
    z_projection = np.max(img[:,:,:], axis = 0)
    #masks_z_projection = np.max(masks[:,:,:], axis = 0)
    features_filt_01 = filter_spots(df_features, measure = measure, signal_strength = thresholds[0])
    features_filt_02 = filter_spots(df_features, measure = measure, signal_strength = thresholds[1])
    features_filt_03 = filter_spots(df_features, measure = measure, signal_strength =  thresholds[2])
    features_filt_04 = filter_spots(df_features, measure = measure, signal_strength = thresholds[3])


    fig, axs = plt.subplots(2,2,figsize = (6,7),dpi= 450)

    for axss in axs:
        for ax in axss:
            ax.set_xticks([])
            ax.set_yticks([])


    axs[0,0].imshow(z_projection, cmap = 'gray_r')
    axs[0,0].scatter(features_filt_01[['x']],features_filt_01[['y']], s = 0.3, linewidths=0.1, alpha = 1, edgecolors = 'red', facecolors='none')
    #axs[0,0].contour(masks_z_projection, [0.5], linewidths=0.5, colors='green')
    axs[0,0].set_title(f'{measure}>{thresholds[0]} (n={len(features_filt_01)})', fontsize=8,y=-0.01)

    axs[0,1].imshow(z_projection, cmap = 'gray_r')
    axs[0,1].scatter(features_filt_02[['x']],features_filt_02[['y']], s = 0.3, linewidths=0.1, alpha = 1, edgecolors = 'red', facecolors='none')
    #axs[0,1].contour(masks_z_projection, [0.5], linewidths=0.5, colors='green')
    axs[0,1].set_title(f'{measure}>{thresholds[1]} (n={len(features_filt_02)})', fontsize=8,y=-0.01)

    axs[1,0].imshow(z_projection, cmap = 'gray_r')
    axs[1,0].scatter(features_filt_03[['x']],features_filt_03[['y']], s = 0.3, linewidths=0.11, alpha = 1, edgecolors = 'red', facecolors='none')
    #axs[1,0].contour(masks_z_projection, [0.5], linewidths=0.5, colors='green')
    axs[1,0].set_title(f'{measure}>{thresholds[2]} (n={len(features_filt_03)})', fontsize=8,y=-0.01)

    axs[1,1].imshow(z_projection, cmap = 'gray_r')
    axs[1,1].scatter(features_filt_04[['x']],features_filt_04[['y']], s = 0.3, linewidths=0.1, alpha = 1, edgecolors = 'red', facecolors='none')
    #axs[1,1].contour(masks_z_projection, [0.5], linewidths=0.5, colors='green')
    axs[1,1].set_title(f'{measure}>{thresholds[3]} (n={len(features_filt_04)})', fontsize=8,y=-0.01)

    plt.tight_layout()
    fig.suptitle(f'Spot detection for {df.filename.iloc[index]}', fontsize=10)
    #xs[0,0].imshow(masks[10,:,:]>0,cmap = 'gray_r')

    if display_plot:
        plt.show()
    else:
        fig.savefig(output_path + 'qc/spots_'+df.id.iloc[index]+'.pdf')
        plt.close()


In [98]:
def plot_qc_spot_threshold(index, df, display_plot=False, measure='signal'):
    ''' Fits mixed gaussian model to find threshold of spot mass

    Parameters:
    index (int): Index of image whose spots should be analysed
    df (pd.DataFrame): Data frame with list of image ids

    Returns:
    Saves a spotThreshold__.pdf for each image with a histogram, model fit and 
    threshold and returns image id, number of spots and the estimated threshold
    '''
    df_spots = pd.read_csv(output_path+'spots/'+df.id.iloc[index]+'.csv')
    x = np.array(df_spots[measure]).reshape(-1,1)
    gm =GaussianMixture(n_components=2,random_state=0).fit(x)
    mu1=gm.means_[0]
    mu2=gm.means_[1]
    sigma1=np.sqrt(gm.covariances_[0])
    sigma2=np.sqrt(gm.covariances_[1])
    threshold=np.round(float(mu1+3*sigma1),2)

    x_fit = np.linspace(0,max(x),100)
    y_fit = gm.score_samples(x_fit)

    plt.figure(figsize=(10,6))
    g = sns.histplot(data=df_spots,x=measure,stat='density',label="Data")
    plt.plot(x_fit,np.exp(y_fit),color='red',lw=1,ls='-',label="Fitted bimodal distribution")
    plt.axvline(mu1,0,1,color='red',lw=0.5,ls="--",label="Mean first gaussian")
    plt.axvline(threshold,0,1,color='red',lw=2,ls="-",label="Threshold (mean1+3*SD1)")
    plt.axvline(mu2,0,1,color='red',lw=0.5,ls=":",label="Mean second gaussian")
    plt.annotate('Threshold='+str(threshold),xy=(0.4,0.9),xycoords='axes fraction')
    plt.title('Distribution of spot '+measure+' as mixture of two gaussians')
    plt.xlabel(measure)
    plt.legend()

    if display_plot:
        plt.show()
    else:
        plt.savefig(output_path + 'qc/spotGMM_'+df.id.iloc[index]+'.pdf')
        plt.close()
    id=df.id.iloc[index]
    num_spot=len(df_spots)
    return(id, num_spot, threshold)


In [99]:
def run_spot_analysis(indices, df, useRaw=True, diameter=(7,9,9), separation=(5,7,7), qc_thresholds=[1,2.5,5,10]):
    '''Finds spots and estimates threshold'''
    ids = list()
    num_spots = list()
    thresholds = list()
    for index in tqdm.tqdm(indices):
        # Load the data
        # with ND2Reader(df.filepath.iloc[index]) as images:
        #     images.bundle_axes = ['z','x','y','c']
        #     img = images[0]
        if useRaw:
            img = BioImage(df.raw_filepath.iloc[index], reader=bioio_nd2.Reader)
            img = img.get_image_data("ZYX", T=0, C=spotChannel)
        else:
            img = BioImage(df.denoised_filepath.iloc[index], reader=bioio_tifffile.Reader)
            img = img.get_image_data("ZYX", T=0, C=spotChannel)

        # Run t
        masks = BioImage(output_path+'segmentation/'+df.id.iloc[index]+'.tif', reader=bioio_tifffile.Reader)
        masks = masks.get_image_data("ZYX", T=0, C=0)

        masks_edt = BioImage(output_path+'edt/'+df.id.iloc[index]+'.tif', reader=bioio_tifffile.Reader)
        masks_edt = masks_edt.get_image_data("ZYX", T=0, C=0)

        # Find the spots (location given in image coordinate system )
        df_spots = find_spots(img[:,:,:],diameter = diameter,separation = separation)
        df_features = extract_spot_features(df_spots,masks,masks_edt)    # For all the spots calculate the features

        
        df_features = filter_spots(df_features, measure='signal', signal_strength = 0.0001)
        if len(df_features) == 0:
            print('EMPTY DF!!')
            print(df.filename.iloc[index])
            print('INDEX:' + str(index))
        else:
            print('found spots:' + str(len(df_features)))


        df_features.loc[:,'id'] = df.id.iloc[index]

        df_features.to_csv(output_path+'spots/'+df.id.iloc[index]+'.csv') # Back up the DF for this FOV
        
        #plot spots detected at different thresholds
        plot_qc_spot_detection(index, df, df_features, img, display_plot = False, measure = 'signal', thresholds = qc_thresholds) # Plot and save the QC of the spot detection

        # Use mixed Guassian model to separate background from true spots
        id, num_spot, threshold = plot_qc_spot_threshold(index, df, display_plot = False, measure = 'signal')
        ids.append(id)
        num_spots.append(num_spot)
        thresholds.append(threshold)
    
    df_thresholds = pd.DataFrame(data = {'id': ids, 'num_spots': num_spots, 'threshold':thresholds })
    df_thresholds.to_csv(output_path+'spotGMMthresholds.csv',index=False)
    return df_thresholds


def replot_spots_with_thresholds(indices, df, useRaw=True, qc_thresholds=[1,2.5,5,10]):
    '''Replots the spot qc images without recalculating the spots, so one can try different qc thresholds'''
    for index in tqdm.tqdm(indices):
        # Load the data
        # with ND2Reader(df.filepath.iloc[index]) as images:
        #     images.bundle_axes = ['z','x','y','c']
        #     img = images[0]
        if useRaw:
            img = BioImage(df.raw_filepath.iloc[index], reader=bioio_nd2.Reader)
            img = img.get_image_data("ZYX", T=0, C=spotChannel)
        else:
            img = BioImage(df.denoised_filepath.iloc[index], reader=bioio_tifffile.Reader)
            img = img.get_image_data("ZYX", T=0, C=spotChannel)

        # Run t
        masks = BioImage(output_path+'segmentation/'+df.id.iloc[index]+'.tif', reader=bioio_tifffile.Reader)
        masks = masks.get_image_data("ZYX", T=0, C=0)

        df_features =  pd.read_csv(output_path+'spots/'+df.id.iloc[index]+'.csv') # Back up the DF for this FOV
        
        #plot spots detected at different thresholds
        plot_qc_spot_detection(index, df, df_features, img, display_plot = False, measure = 'signal', thresholds = qc_thresholds) # Plot and save the QC of the spot detection


## Functions to gather results into single file

In [100]:
def collect_nuclear_segementation_data(indices, df, suffix = 'v001'):
    '''Collects nuclear intensity and intensity vs distance data for all nuclei in the dataset'''
    df_nuclei = pd.DataFrame()
    for index in tqdm.tqdm(indices):
        df_tmp = pd.read_csv(output_path+'nuclei/'+df.id.iloc[index]+'.csv')
        df_nuclei = pd.concat([df_nuclei,df_tmp])
    df_nuclei.to_csv(output_path+'nuclei_analysis_'+suffix+'.csv',index=False)



def collect_nuclear_distance_data(indices, df, suffix = 'v001'):
    '''Collects nuclear intensity and intensity vs distance data for all nuclei in the dataset'''
    df_dist = pd.DataFrame()
    for index in tqdm.tqdm(indices):
        df_tmp = pd.read_pickle(output_path+'dist/'+df.id.iloc[index]+'.pkl')
        df_dist = pd.concat([df_dist,df_tmp])
    df_dist.to_pickle(output_path+'dist_analysis_'+suffix+'.pkl')


def collect_spot_data(indices, df, suffix = 'v001'):
    '''Collects spot data for all images'''
    df_spots = pd.DataFrame()
    for index in tqdm.tqdm(indices):
        df_tmp = pd.read_csv(output_path+'spots/'+df.id.iloc[index]+'.csv')
        df_spots = pd.concat([df_spots,df_tmp])
    df_spots.to_csv(output_path+'spots_analysis_'+suffix+'.csv',index=False)


## Running the analysis for nuclear segmentation

In [101]:
# run analysis to segment nuclei
indices=range(0,len(df))
#indices=[0]

#run_nuclear_segmentation(indices, df) 

run_dist_analysis(indices,df)

#collect_nuclear_segementation_data(indices, df, suffix = 'v001')
collect_nuclear_distance_data(indices, df, suffix = 'v001')

  0%|          | 0/24 [00:00<?, ?it/s]

/mnt/external.data/MeisterLab/Dario/Imaging/DPY27/1268/20240915_3d/20240915_1268_E_bean_15um.nd2


/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)
/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/skimage/measure/_regionprops.py:433: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1575294842  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 30  Error-roundoff 3.8e-14  _one-merge 2.7e-13
  _near-inside 1.3e-12  Visible-distance 7.7e-14  U-max-coplanar 7.7e-14
  Width-outside 1.5e-13  _wide-facet 4.6e-13  _maxoutside 3.1e-13

  return convex_hull_image(self.image)
/home/jsemple/miniforge3/envs/lhcellpose1/lib/py

/mnt/external.data/MeisterLab/Dario/Imaging/DPY27/1268/20240915_3d/20240915_1268_E_bean_15um_02.nd2


/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)
/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/skimage/measure/_regionprops.py:433: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1575815859  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 25  Error-roundoff 3.5e-14  _one-merge 2.4e-13
  _near-inside 1.2e-12  Visible-distance 6.9e-14  U-max-coplanar 6.9e-14
  Width-outside 1.4e-13  _wide-facet 4.2e-13  _maxoutside 2.8e-13

  return convex_hull_image(self.image)
/home/jsemple/miniforge3/envs/lhcellpose1/lib/py

...

/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)
/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/skimage/measure/_regionprops.py:433: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1587412689  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 26  Error-roundoff 3.6e-14  _one-merge 2.5e-13
  _near-inside 1.3e-12  Visible-distance 7.2e-14  U-max-coplanar 7.2e-14
  Width-outside 1.4e-13  _wide-facet 4.3e-13  _maxoutside 2.9e-13

  return convex_hull_image(self.image)
/home/jsemple/miniforge3/envs/lhcellpose1/lib/py

/mnt/external.data/MeisterLab/Dario/Imaging/DPY27/1268/20240915_3d/20240915_1268_E_mid_15um_07.nd2


/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)
/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/skimage/measure/_regionprops.py:433: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1587782443  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 30  Error-roundoff 4.2e-14  _one-merge 2.9e-13
  _near-inside 1.5e-12  Visible-distance 8.3e-14  U-max-coplanar 8.3e-14
  Width-outside 1.7e-13  _wide-facet 5e-13  _maxoutside 3.3e-13

  return convex_hull_image(self.image)
/home/jsemple/miniforge3/envs/lhcellpose1/lib/pyth

/mnt/external.data/MeisterLab/Dario/Imaging/DPY27/1268/20240915_3d/20240915_1268_E_mid_15um_08.nd2


/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)
/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/skimage/measure/_regionprops.py:433: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1588269846  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 35  Error-roundoff 4.9e-14  _one-merge 3.4e-13
  _near-inside 1.7e-12  Visible-distance 9.7e-14  U-max-coplanar 9.7e-14
  Width-outside 1.9e-13  _wide-facet 5.8e-13  _maxoutside 3.9e-13

  return convex_hull_image(self.image)
/home/jsemple/miniforge3/envs/lhcellpose1/lib/py

## Running the analysis for spot detection

In [18]:
#indices=[0]
indices=range(0,len(df))

run_spot_analysis(indices, df, useRaw=True, diameter=(7,9,9), separation=(7,9,9), qc_thresholds=[10,15,20,25])
#replot_spots_with_thresholds(indices, df, useRaw=False, qc_thresholds=[10, 15, 20, 25])

collect_spot_data(indices, df, suffix = 'v001')

  0%|          | 0/24 [00:00<?, ?it/s]/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


found spots:1573


/tmp/ipykernel_1991396/3139461931.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features.loc[:,'id'] = df.id.iloc[index]
/tmp/ipykernel_1991396/450708869.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  threshold=np.round(float(mu1+3*sigma1),2)
  4%|▍         | 1/24 [00:15<05:47, 15.13s/it]/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


found spots:1736


/tmp/ipykernel_1991396/3139461931.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features.loc[:,'id'] = df.id.iloc[index]
/tmp/ipykernel_1991396/450708869.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  threshold=np.round(float(mu1+3*sigma1),2)
  8%|▊         | 2/24 [00:30<05:36, 15.30s/it]/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


...

/tmp/ipykernel_1991396/3139461931.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features.loc[:,'id'] = df.id.iloc[index]
/tmp/ipykernel_1991396/450708869.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  threshold=np.round(float(mu1+3*sigma1),2)
 92%|█████████▏| 22/24 [06:53<00:37, 18.53s/it]/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


found spots:2097


/tmp/ipykernel_1991396/3139461931.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features.loc[:,'id'] = df.id.iloc[index]
/tmp/ipykernel_1991396/450708869.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  threshold=np.round(float(mu1+3*sigma1),2)
 96%|█████████▌| 23/24 [07:11<00:18, 18.34s/it]/home/jsemple/miniforge3/envs/lhcellpose1/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


found spots:1083


/tmp/ipykernel_1991396/3139461931.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features.loc[:,'id'] = df.id.iloc[index]
/tmp/ipykernel_1991396/450708869.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  threshold=np.round(float(mu1+3*sigma1),2)
100%|██████████| 24/24 [00:00<00:00, 151.07it/s]
